import the required libraries 

In [ ]:
import pypyodbc as odbc
import pandas as pd
from sqlalchemy import create_engine
import urllib

connect and loading to the server

In [ ]:
DRIVER_NAME='ODBC Driver 17 for SQL Server'
SERVER_NAME='DESKTOP-VRHL0RA\MSSQLSERVER1'
DATABASE_NAME='Customers'
username='sa'
password='Dheepansh@2810'

# connection_string=f""" 
# DRIVER={{{DRIVER_NAME}}};
# SERVER={SERVER_NAME};
# DATABASE={DATABASE_NAME};
# uid={username};
# pwd={password};
# Trust_Connection=yes;
# """
# conn=odbc.connect(connection_string)
# print(conn)
# Encode parameters for SQLAlchemy
params = urllib.parse.quote_plus(
    f"DRIVER={{{DRIVER_NAME}}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};UID={username};PWD={password}"
)
 
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

 # Create engine

customer = pd.read_csv("cleaned_us_customer.csv")
transactions=pd.read_csv("cleaned_transactions.csv")
orders=pd.read_csv("order_data 1.csv")

customer.to_sql('Customers', con=engine, if_exists='replace', index=False)
transactions.to_sql('Transactions', con=engine, if_exists='replace', index=False)
orders.to_sql('Orders', con=engine, if_exists='replace', index=False)



read data from sql server

In [ ]:
read_customer_sql=pd.read_sql_table('Customers',con=engine)
read_order_sql=pd.read_sql_table('Orders',con=engine)


merge the required tables

In [ ]:
unified_view=pd.merge(read_customer_sql,read_order_sql,on='customer_id')

split the name into first name and last name, remove suffixes in lastname

In [ ]:

unified_view[['firstname','lastname']]=unified_view['name'].str.split(' ',n=1,expand=True)
unified_view['lastname'] = unified_view['lastname'].str.split(' ').str[0]
 

mapping the loyalty status to numerical thresholds

In [ ]:
customer_classfication={'Silver':1,'Gold':2,'Bronze':0}
unified_view['mapping_value']=unified_view['loyalty_status'].map(customer_classfication)

In [ ]:
unified_view

reorder the columns firstname and lastname

In [ ]:
cols=unified_view.columns.tolist()
cols.remove('firstname')
cols.insert(2,'firstname')
unified_view=unified_view[cols]
print(unified_view)

cols.remove('lastname')
cols.insert(3,'lastname')
unified_view=unified_view[cols]

upload to sqlserver

In [ ]:
unified_view.to_sql('customers_unified_orders',con=engine,if_exists='replace',index=False)

In [ ]:
query="""select * from customers_unified_orders"""
inf=pd.read_sql_query(query,con=engine)
inf.head()
